# EIF4A2

data derived from [Paul MS, et al. (2023) Rare EIF4A2 variants are associated with a neurodevelopmental disorder characterized by intellectual disability, hypotonia, and epilepsy. Am J Hum Genet. 2023 Jan 5;110(1):120-145. PMID: 36528028](https://pubmed.ncbi.nlm.nih.gov/36528028/)

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import IndividualTable, QcVisualizer, HpoaTableBuilder
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.33


In [2]:
PMID="PMID:36528028"
title = "Rare EIF4A2 variants are associated with a neurodevelopmental disorder characterized by intellectual disability, hypotonia, and epilepsy"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by = "ORCID:0000-0002-0736-9199"
metadata = MetaData(created_by=created_by, citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")


HPO version 2024-01-16


In [3]:
df = pd.read_excel("input/EIF4A2_Paul_2023.xlsx")

In [4]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['individual_id'] = dft.index  # Set the new column 'patient_id' to be identical to the contents of the index
dft.head(2) # check the transposed table

Individual #,EIF4A2 variant information,Inheritance,Genomic (GRCh38),allele_1,allele_2,cDNA (GenBank: NM_001967.3),Protein,Sequencing method,gnomAD frequency,CADD score,...,Feeding issues,Dysmotility,Hypoventilation,Vision/eye abnormalities,Hearing impairment,Dysmorphic features,Congenital anomalies,Muscle findings (biopsy results if applicable),Other,individual_id
1,NaN,de novo,"chr3: 186,783,615C:G",c.5C>G,na,c.5C>G,p.Ser2Cys,"research-based exome sequencing (trio), Sanger confirmed",3.976 × 10−6,29.7,...,–,–,–,–,–,"epicanthal folds, small ears, anteverted nares, small mouth",–,ND,–,1
2,NaN,autosomal recessive,"chr3: 186,784,592_186,502,381TTGA:T",c.109_111del,na,c.109_111del,p.Asp37del,research-based exome sequencing (single),3.976 × 10−6,23.1,...,–,–,–,–,–,"microcephaly, short stature",–,ND,ataxia,2


In [5]:
column_mapper_list = list()
#res = OptionColumnMapper.autoformat(df=dft, hpo_cr=hpo_cr)
#print(res)

In [6]:
delayed_speech_development_d = {'+': 'Delayed speech and language development',
 'absent speech': 'Absent speech'}
excluded = {}
delayed_speech_developmentMapper = OptionColumnMapper(column_name="Delayed speech development", concept_recognizer=hpo_cr, option_d=delayed_speech_development_d, excluded_d=excluded)
column_mapper_list.append(delayed_speech_developmentMapper)
delayed_speech_developmentMapper.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Delayed speech and language development (HP:0000750) (observed)",6
1,"original value: ""+, absent speech"" -> HP: Delayed speech and language development (HP:0000750) (observed)",8


In [7]:
delayed_gross_motor_development_d = {'+': 'Delayed gross motor development',
 'severe': 'Delayed gross motor development',
 'does not sit': ["Delayed gross motor development",'Delayed ability to sit'],
 'spastic tetraparesis': 'Spastic tetraparesis'}
excluded = {}
delayed_gross_motor_developmentMapper = OptionColumnMapper(column_name="Delayed gross motor development", concept_recognizer=hpo_cr, option_d=delayed_gross_motor_development_d, excluded_d=excluded)
column_mapper_list.append(delayed_gross_motor_developmentMapper)
delayed_gross_motor_developmentMapper.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Delayed gross motor development (HP:0002194) (observed)",10
1,"original value: ""+, severe, does not sit"" -> HP: Delayed gross motor development (HP:0002194) (observed)",1
2,"original value: ""+, severe"" -> HP: Delayed gross motor development (HP:0002194) (observed)",1
3,"original value: ""+, spastic tetraparesis"" -> HP: Delayed gross motor development (HP:0002194) (observed)",1


In [8]:
delayed_fine_motor_skills_d = {
 '+': 'Delayed fine motor development',
 'severe': 'Delayed fine motor development'}
excluded = {}
delayed_fine_motor_skillsMapper = OptionColumnMapper(column_name="Delayed fine motor skills", concept_recognizer=hpo_cr, option_d=delayed_fine_motor_skills_d, excluded_d=excluded)
column_mapper_list.append(delayed_fine_motor_skillsMapper)
delayed_fine_motor_skillsMapper.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Delayed fine motor development (HP:0010862) (observed)",10
1,"original value: ""+, severe"" -> HP: Delayed fine motor development (HP:0010862) (observed)",1


In [9]:
gdd_d = {'GDD': 'Global developmental delay',
 'borderline/mild ID (IQ 78)': 'PLACEHOLDER',
 'mild ID': 'Intellectual disability, mild',
# 'mild FSIQ 87': 'PLACEHOLDER',
 'severe neurodegenerative disease since infancy; severe ID': 'Intellectual disability, severe',
 'no IQ measurable': 'Intellectual disability, severe',
 'mild GDD': 'Mild global developmental delay',
 'severe GDD': 'Severe global developmental delay',
 'severe ID': 'Intellectual disability, severe',
 'ID': 'Intellectual disability',
 'severe': 'Intellectual disability, severe'}
excluded = {}
gddMapper = OptionColumnMapper(column_name="Developmental delay/intellectual disability (IQ if available)", concept_recognizer=hpo_cr, option_d=gdd_d, excluded_d=excluded)
column_mapper_list.append(gddMapper)
gddMapper.preview_column(dft)


,mapping,count
0,"original value: ""GDD, borderline/mild ID (IQ 78)"" -> HP: Global developmental delay (HP:0001263) (observed)",1
1,"original value: ""mild ID"" -> HP: Intellectual disability, mild (HP:0001256) (observed)",1
2,"original value: ""GDD, mild FSIQ 87"" -> HP: Global developmental delay (HP:0001263) (observed)",1
3,"original value: ""severe neurodegenerative disease since infancy; severe ID, no IQ measurable"" -> HP: Neurodegeneration (HP:0002180) (observed)",2
4,"original value: ""mild GDD"" -> HP: Mild global developmental delay (HP:0011342) (observed)",1
5,"original value: ""severe GDD"" -> HP: Severe global developmental delay (HP:0011344) (observed)",2
6,"original value: ""severe ID"" -> HP: Intellectual disability, severe (HP:0010864) (observed)",1
7,"original value: ""GDD"" -> HP: Global developmental delay (HP:0001263) (observed)",3
8,"original value: ""severe GDD, ID"" -> HP: Severe global developmental delay (HP:0011344) (observed)",1
9,"original value: ""severe, GDD"" -> HP: Intellectual disability, severe (HP:0010864) (observed)",1


In [10]:
seizures_d = {'+': 'Seizure',
 'myoclonic seizures starting at 1 year': 'Myoclonic seizure',
 'myoclonic seizures': 'Myoclonic seizure',
 'generalized-tonic clonic': 'Bilateral tonic-clonic seizure',
 'onset 19\xa0months': 'Seizure',
 'starting at 3\xa0months of age; epileptic spasms': 'Epileptic spasm',
 'tonic seizures with clonic component': 'Seizure',
 'Lennox Gastaut syndrome with tonic seizures and atypical absence seizures': 'Atypical absence seizure',
 'severe and present in infancy': 'Seizure',
 'focal': 'Focal-onset seizure',
 'well controlled; starting at 3 years': 'Seizure',
 'West syndrome (infantile spasms': 'Infantile spasms',
 'onset 6months)': 'Seizure',
 'starting at 5\xa0months: infantile spasms and myoclonic seizures': ['Infantile spasms','Myoclonic seizure'],
 'diagnosed with West syndrome; now with tonic': 'Epileptic spasm',
 'gelastic': 'Focal emotional seizure with laughing',
 'and absence seizures and diagnosed with Lennox-Gastaut syndrome': 'Generalized non-motor (absence) seizure',
 }
excluded = {}
seizuresMapper = OptionColumnMapper(column_name="Seizures", concept_recognizer=hpo_cr, option_d=seizures_d, excluded_d=excluded)
column_mapper_list.append(seizuresMapper)
seizuresMapper.preview_column(dft)

,mapping,count
0,"original value: ""+, myoclonic seizures starting at 1 year"" -> HP: Seizure (HP:0001250) (observed)",1
1,"original value: ""+, myoclonic seizures"" -> HP: Seizure (HP:0001250) (observed)",1
2,"original value: ""+, generalized-tonic clonic, onset 19 months"" -> HP: Seizure (HP:0001250) (observed)",1
3,"original value: ""starting at 3 months of age; epileptic spasms, tonic seizures with clonic component"" -> HP: Epileptic spasm (HP:0011097) (observed)",1
4,"original value: ""+"" -> HP: Seizure (HP:0001250) (observed)",1
5,"original value: ""Lennox Gastaut syndrome with tonic seizures and atypical absence seizures"" -> HP: Atypical absence seizure (HP:0007270) (observed)",1
6,"original value: ""+, severe and present in infancy, sitting position never acquired"" -> HP: Seizure (HP:0001250) (observed)",1
7,"original value: ""+, focal, well controlled; starting at 3 years"" -> HP: Seizure (HP:0001250) (observed)",1
8,"original value: ""+, West syndrome (infantile spasms, onset 6months)"" -> HP: Seizure (HP:0001250) (observed)",1
9,"original value: ""starting at 5 months: infantile spasms and myoclonic seizures, diagnosed with West syndrome; now with tonic, focal, gelastic, and absence seizures and diagnosed with Lennox-Gastaut syndrome"" -> HP: Infantile spasms (HP:0012469) (observed)",1


In [11]:
autism = SimpleColumnMapper(column_name="Autism", hpo_id="HP:0000717", hpo_label="Autism", observed="+", excluded="-")
column_mapper_list.append(autism)
autism.preview_column(dft)

,mapping,count
0,"original value: ""–"" -> HP: Autism (HP:0000717) (not measured)",11
1,"original value: ""+"" -> HP: Autism (HP:0000717) (observed)",3
2,"original value: ""not tested"" -> HP: Autism (HP:0000717) (not measured)",1


In [12]:
structural_brain_abnormalities_on_mri_d = {
 'rotated small cerebellar hemispheres rotated small vermis': 'Cerebellar vermis hypoplasia',
 'small pons': 'Hypoplasia of the pons',
 'plagiocephaly': 'Plagiocephaly',
  'increasing size of cisterna magna': 'Enlarged cisterna magna',
 'decreased volume of corpus callosum': 'Thin corpus callosum',
 'prominent ventricles': 'Ventriculomegaly',
 'retrocerebellar arachnoid cysts': 'Arachnoid cyst',
 'short corpus callosum with disproportionately small splenium': 'Short corpus callosum',
 'thinned corpus callosum': 'Thin corpus callosum',
 'enlarged lateral and 3rd ventricles': 'Ventriculomegaly',
 'gliosis in the occipital periventricular white matter': 'Gliosis',
 'small optic nerves': 'Optic nerve hypoplasia',
 'hypoplastic cerebellar vermis': 'Cerebellar vermis hypoplasia',
 'thin corpus callosum': 'Thin corpus callosum',
 'hypoplastic pons': 'Hypoplasia of the pons'}
excluded = {}
structural_brain_abnormalities_on_mriMapper = OptionColumnMapper(column_name="Structural brain abnormalities on MRI", concept_recognizer=hpo_cr, option_d=structural_brain_abnormalities_on_mri_d, excluded_d=excluded)
column_mapper_list.append(structural_brain_abnormalities_on_mriMapper)
structural_brain_abnormalities_on_mriMapper.preview_column(dft)

,mapping,count
0,"original value: ""small anterior commissure, subtle interdigitation of interomedial frontal gyri, rotated small cerebellar hemispheres rotated small vermis, small pons, medulla and middle cerebellar peduncles, small rounded hippocampi, plagiocephaly, progressive mild cerebral volume loss, slightly small optic chiasm and tracts, increasing size of cisterna magna"" -> HP: Cerebellar vermis hypoplasia (HP:0001320) (observed)",1
1,"original value: ""decreased volume of corpus callosum, small anterior commissure, prominent ventricles, subtle interdigitation of interomedial frontal gyri, rotated small cerebellar hemispheres rotated small vermis, small pons, medulla and middle cerebellar peduncles, small rounded hippocampi, plagiocephaly, small temporal tips, uncovered insula"" -> HP: Thin corpus callosum (HP:0033725) (observed)",1
2,"original value: ""retrocerebellar arachnoid cysts"" -> HP: Arachnoid cyst (HP:0100702) (observed)",1
3,"original value: ""short corpus callosum with disproportionately small splenium"" -> HP: Short corpus callosum (HP:0200012) (observed)",1
4,"original value: ""decreased volume of corpus callosum, small anterior commissure, subtle interdigitation of the inferomedial frontal gyri"" -> HP: Thin corpus callosum (HP:0033725) (observed)",1
5,"original value: ""thinned corpus callosum, enlarged lateral and 3rd ventricles"" -> HP: Thin corpus callosum (HP:0033725) (observed)",1
6,"original value: ""small temporal tips and uncovered insula, decreased volume but fully formed corpus callosum; small anterior commissure, decreased cerebral volume predominantly white matter and most prominent right greater than left frontal lobe and peritrigonal regions with probably secondary decreased thalamic and cerebral peduncle volume, gliosis in the occipital periventricular white matter, interdigitation of the inferomedial frontal gyri, small fornices, small optic nerves, chiasm and tracts, mildly rotated, slightly small vermis, mild prominence of the cerebellar and vermian fissures suggesting gray matter volume loss, small pons, medulla and middle cerebellar peduncles"" -> HP: Gliosis (HP:0002171) (observed)",1
7,"original value: ""hypoplastic cerebellar vermis, thin corpus callosum, hypoplastic pons"" -> HP: Cerebellar vermis hypoplasia (HP:0001320) (observed)",1


In [13]:
hypotonia_d = {'+': 'Hypotonia',
 'mild': 'Hypotonia',
 'history of hypotonia': 'Hypotonia',
 'now possible hypertonia': 'Hypertonia',
 'diffuse': 'Hypotonia',
 'severe and present in infancy': 'Hypotonia',
 'sitting position never acquired': 'Hypotonia',
 'central': 'Hypotonia',
 'present in infancy': 'Hypotonia',
 'now with hypertonia of extremities': 'Hypertonia',
 'severe': 'Hypotonia'}
excluded = {}
hypotoniaMapper = OptionColumnMapper(column_name="Hypotonia", concept_recognizer=hpo_cr, option_d=hypotonia_d, excluded_d=excluded)
column_mapper_list.append(hypotoniaMapper)
hypotoniaMapper.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Hypotonia (HP:0001252) (observed)",7
1,"original value: ""+, mild"" -> HP: Hypotonia (HP:0001252) (observed)",1
2,"original value: ""+, history of hypotonia, now possible hypertonia"" -> HP: Hypotonia (HP:0001252) (observed)",1
3,"original value: ""+, diffuse"" -> HP: Hypotonia (HP:0001252) (observed)",1
4,"original value: ""+, severe and present in infancy, sitting position never acquired"" -> HP: Hypotonia (HP:0001252) (observed)",1
5,"original value: ""+, central"" -> HP: Hypotonia (HP:0001252) (observed)",1
6,"original value: ""present in infancy, now with hypertonia of extremities"" -> HP: Hypotonia (HP:0001252) (observed)",1
7,"original value: ""+, severe"" -> HP: Hypotonia (HP:0001252) (observed)",1


In [14]:
hypoventilation_d = {'+': 'Hypoventilation',
 'nasal cannula while asleep': 'Hypoventilation',
 'tracheostomy in place': 'Hypoventilation',
 'tracheostomy': 'Hypoventilation'}
excluded = {}
hypoventilationMapper = OptionColumnMapper(column_name="Hypoventilation", concept_recognizer=hpo_cr, option_d=hypoventilation_d, excluded_d=excluded)
column_mapper_list.append(hypoventilationMapper)
hypoventilationMapper.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Hypoventilation (HP:0002791) (observed)",1
1,"original value: ""+, nasal cannula while asleep"" -> HP: Hypoventilation (HP:0002791) (observed)",1
2,"original value: ""+, tracheostomy in place"" -> HP: Hypoventilation (HP:0002791) (observed)",1
3,"original value: ""+, tracheostomy"" -> HP: Hypoventilation (HP:0002791) (observed)",1


In [15]:
behavior_abnormalities_d = {
 'impulsivity': 'Impulsivity',
 #'inattention': 'PLACEHOLDER',
 'ADHD': 'Attention deficit hyperactivity disorder',
 'head banging': 'Head-banging',
 #'repetitive motions': 'PLACEHOLDER',
 'very limited interactions': 'Decreased social interactions',
}
excluded = {}
behavior_abnormalitiesMapper = OptionColumnMapper(column_name="Behavior abnormalities", concept_recognizer=hpo_cr, option_d=behavior_abnormalities_d, excluded_d=excluded)
column_mapper_list.append(behavior_abnormalitiesMapper)
behavior_abnormalitiesMapper.preview_column(dft)


,mapping,count
0,"original value: ""+, impulsivity, inattention, ADHD"" -> HP: Impulsivity (HP:0100710) (observed)",1
1,"original value: ""head banging, repetitive motions"" -> HP: Head-banging (HP:0012168) (observed)",1
2,"original value: ""very limited interactions"" -> HP: Decreased social interactions (HP:5200310) (observed)",1


In [16]:
failure_to_thrive_d = {'+': 'Failure to thrive',
 'early': 'Failure to thrive',
 'in infancy': 'Failure to thrive'}
excluded = {'-':"Failure to thrive"}
failure_to_thriveMapper = OptionColumnMapper(column_name="Failure to thrive", concept_recognizer=hpo_cr, option_d=failure_to_thrive_d, excluded_d=excluded)
column_mapper_list.append(failure_to_thriveMapper)
failure_to_thriveMapper.preview_column(dft)

,mapping,count
0,"original value: ""+, early"" -> HP: Failure to thrive (HP:0001508) (observed)",1
1,"original value: ""+, in infancy"" -> HP: Failure to thrive (HP:0001508) (observed)",2
2,"original value: ""+"" -> HP: Failure to thrive (HP:0001508) (observed)",1


In [17]:
feeding_issues_d = {'+': 'Feeding difficulties',
 'G-tube': 'Tube feeding',
 'GER': 'Tube feeding',
 'hx of NG after birth': 'Tube feeding',
 'no suckling reflex': 'Poor suck',
 'G- and J-tubes': 'Tube feeding'}
excluded = {}
feeding_issuesMapper = OptionColumnMapper(column_name="Feeding issues", concept_recognizer=hpo_cr, option_d=feeding_issues_d, excluded_d=excluded)
column_mapper_list.append(feeding_issuesMapper)
feeding_issuesMapper.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Feeding difficulties (HP:0011968) (observed)",3
1,"original value: ""+, G-tube"" -> HP: Feeding difficulties (HP:0011968) (observed)",5
2,"original value: ""G-tube, GER"" -> HP: Tube feeding (HP:0033454) (observed)",1
3,"original value: ""+, hx of NG after birth"" -> HP: Feeding difficulties (HP:0011968) (observed)",1
4,"original value: ""+, no suckling reflex, G- and J-tubes"" -> HP: Feeding difficulties (HP:0011968) (observed)",1


In [18]:
dysmotility_d = {
 'difficulty swallowing': 'Dysphagia',
 'constipation': 'Constipation',
 'dysphagia': 'Dysphagia',
 'feeding difficulties': 'Feeding difficulties'}
excluded = {}
dysmotilityMapper = OptionColumnMapper(column_name="Dysmotility", concept_recognizer=hpo_cr, option_d=dysmotility_d, excluded_d=excluded)
column_mapper_list.append(dysmotilityMapper)
dysmotilityMapper.preview_column(dft)

,mapping,count
0,"original value: ""+, difficulty swallowing, constipation"" -> HP: Dysphagia (HP:0002015) (observed)",2
1,"original value: ""+, dysphagia, constipation"" -> HP: Dysphagia (HP:0002015) (observed)",2
2,"original value: ""feeding difficulties, dysphagia"" -> HP: Feeding difficulties (HP:0011968) (observed)",1


In [19]:
vision_eye_abnormalities_d = {'visual impairment since 1 year of age (suspected first at 4\xa0months)': 'Visual impairment',
 'blindness impairment since 6 years of age': 'Blindness',
 'cortical blindness': 'Cerebral visual impairment',
 'bilateral exotropia': 'Exotropia',
 'central blindness': 'Blindness'}
excluded = {}
vision_eye_abnormalitiesMapper = OptionColumnMapper(column_name="Vision/eye abnormalities", concept_recognizer=hpo_cr, option_d=vision_eye_abnormalities_d, excluded_d=excluded)
column_mapper_list.append(vision_eye_abnormalitiesMapper)
vision_eye_abnormalitiesMapper.preview_column(dft)


,mapping,count
0,"original value: ""visual impairment since 1 year of age (suspected first at 4 months)"" -> HP: Visual impairment (HP:0000505) (observed)",1
1,"original value: ""blindness impairment since 6 years of age"" -> HP: Blindness (HP:0000618) (observed)",1
2,"original value: ""cortical blindness"" -> HP: Cerebral visual impairment (HP:0100704) (observed)",2
3,"original value: ""bilateral exotropia"" -> HP: Exotropia (HP:0000577) (observed)",1
4,"original value: ""central blindness"" -> HP: Blindness (HP:0000618) (observed)",1


In [20]:
hearing_impairment_d = {'+': 'Hearing impairment',
 'bilateral deafness': 'Hearing impairment',
 'deaf aid': 'Hearing impairment',
 'since 1 year of age (deaf aid)': 'Hearing impairment'}
excluded = {}
hearing_impairmentMapper = OptionColumnMapper(column_name="Hearing impairment", concept_recognizer=hpo_cr, option_d=hearing_impairment_d, excluded_d=excluded)
column_mapper_list.append(hearing_impairmentMapper)
hearing_impairmentMapper.preview_column(dft)

,mapping,count
0,"original value: ""+, bilateral deafness, deaf aid"" -> HP: Hearing impairment (HP:0000365) (observed)",1
1,"original value: ""+, since 1 year of age (deaf aid)"" -> HP: Hearing impairment (HP:0000365) (observed)",1


In [21]:
dysmorphic_features_d = {'epicanthal folds': 'Epicanthus',
 'small ears': 'Microtia',
 'anteverted nares': 'Anteverted nares',
 'small mouth': 'Narrow mouth',
 'microcephaly': 'Microcephaly',
 'short stature': 'Short stature',
 'mild synophrys': 'Synophrys',
 'mild hirsutism on back': 'Hirsutism',
 'bilateral 5th finger clinodactyly': 'Finger clinodactyly',

 'only secondary due to spasticity': 'Spasticity',
 'long palpebral fissures': 'Long palpebral fissure',
 'small hands and feet': 'Small hand',
 'macrocephaly': 'Macrocephaly',
 'hx sagittal craniosynostosis s/p repair': 'Sagittal craniosynostosis',
 'relative macrocephaly': 'Relative macrocephaly',
 'micrognathia': 'Micrognathia',
 'bilaterally inverted nipples': 'Inverted nipples',
 'epicanthus': 'Epicanthus',
 'upslanting palpebral fissures': 'Upslanted palpebral fissure',
 'broad nasal root': 'Wide nasal bridge',
 'short philtrum': 'Short philtrum',
 }
excluded = {}
dysmorphic_featuresMapper = OptionColumnMapper(column_name="Dysmorphic features", concept_recognizer=hpo_cr, option_d=dysmorphic_features_d, excluded_d=excluded)
column_mapper_list.append(dysmorphic_featuresMapper)
dysmorphic_featuresMapper.preview_column(dft)

,mapping,count
0,"original value: ""epicanthal folds, small ears, anteverted nares, small mouth"" -> HP: Epicanthus (HP:0000286) (observed)",1
1,"original value: ""microcephaly, short stature"" -> HP: Microcephaly (HP:0000252) (observed)",1
2,"original value: ""mild synophrys, mild hirsutism on back, bilateral 5th finger clinodactyly"" -> HP: Synophrys (HP:0000664) (observed)",1
3,"original value: ""no, only secondary due to spasticity"" -> HP: Spasticity (HP:0001257) (observed)",1
4,"original value: ""long palpebral fissures, small hands and feet"" -> HP: Long palpebral fissure (HP:0000637) (observed)",1
5,"original value: ""macrocephaly, hx sagittal craniosynostosis s/p repair"" -> HP: Macrocephaly (HP:0000256) (observed)",1
6,"original value: ""relative macrocephaly"" -> HP: Relative macrocephaly (HP:0004482) (observed)",1
7,"original value: ""microcephaly"" -> HP: Microcephaly (HP:0000252) (observed)",1
8,"original value: ""micrognathia, bilaterally inverted nipples"" -> HP: Micrognathia (HP:0000347) (observed)",1
9,"original value: ""epicanthus, upslanting palpebral fissures, broad nasal root, short philtrum, curved upper lip"" -> HP: Epicanthus (HP:0000286) (observed)",1


In [22]:
congenital_anomalies_d = {'sagittal craniosynostosis s/p repair': 'Sagittal craniosynostosis',
 'penile chordee with phimosis': ['Chordee','Phimosis'],
 'PDA s/p closure': 'Patent ductus arteriosus',
 'ostium secundum ASD': 'Secundum atrial septal defect'}
excluded = {}
congenital_anomaliesMapper = OptionColumnMapper(column_name="Congenital anomalies", concept_recognizer=hpo_cr, option_d=congenital_anomalies_d, excluded_d=excluded)
column_mapper_list.append(congenital_anomaliesMapper)
congenital_anomaliesMapper.preview_column(dft)

,mapping,count
0,"original value: ""sagittal craniosynostosis s/p repair"" -> HP: Sagittal craniosynostosis (HP:0004442) (observed)",1
1,"original value: ""penile chordee with phimosis"" -> HP: Chordee (HP:0000041) (observed)",1
2,"original value: ""PDA s/p closure"" -> HP: Patent ductus arteriosus (HP:0001643) (observed)",1
3,"original value: ""ostium secundum ASD"" -> HP: Secundum atrial septal defect (HP:0001684) (observed)",1


In [23]:
#cDNA (GenBank: NM_001967.3)
vmanager = VariantManager(df=dft, 
                          individual_column_name="individual_id",
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2",
                          cohort_name="EIF4A2", 
                          transcript="NM_001967.3")

In [24]:
vmanager.to_summary()

,status,count,alleles
0,mapped,15,"c.1091G>A, c.1084G>A, c.186_187del, c.131_132delTC, c.5C>G, c.481G>T, c.574G>A, c.647C>T, c.109_111del, c.641C>A, c.728C>T, c.646A>G, c.945_947delCAT, c.1161_1166del, c.1032G>C"
1,unmapped,0,


In [25]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='Sex')
#sexMapper.preview_column(dft["Sex"])

In [26]:
#Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures 	620455 
NEDHSS = Disease(disease_id="OMIM:620455", disease_label="Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures")

individual_column_name = 'individual_id'

encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name=individual_column_name,
                        agemapper=AgeColumnMapper.not_provided(), 
                        sexmapper=sexMapper,
                        metadata=metadata)
encoder.set_disease(NEDHSS)

In [27]:
individuals = encoder.get_individuals()
vmanager.add_variants_to_individuals(individuals)

In [28]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,23
INFORMATION,NOT_MEASURED,12


In [29]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
1 (FEMALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.5C>G (heterozygous),"Delayed speech and language development (HP:0000750); Delayed fine motor development (HP:0010862); Global developmental delay (HP:0001263); Intellectual disability, mild (HP:0001256); Epicanthus (HP:0000286); Microtia (HP:0008551); Anteverted nares (HP:0000463); Narrow mouth (HP:0000160)"
2 (FEMALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.109_111del (heterozygous),"Delayed speech and language development (HP:0000750); Intellectual disability, mild (HP:0001256); Hypotonia (HP:0001252); Microcephaly (HP:0000252); Short stature (HP:0004322)"
3 (FEMALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.131_132del (heterozygous),Delayed gross motor development (HP:0002194); Delayed fine motor development (HP:0010862); Global developmental delay (HP:0001263); Hypotonia (HP:0001252); Impulsivity (HP:0100710); Attention deficit hyperactivity disorder (HP:0007018); Feeding difficulties (HP:0011968); Synophrys (HP:0000664); Hirsutism (HP:0001007); Finger clinodactyly (HP:0040019)
4 (MALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.186_187del (heterozygous) NM_001967.3:c.1161_1166del (heterozygous),"Delayed gross motor development (HP:0002194); Myoclonic seizure (HP:0032794); Dysphagia (HP:0002015); Neurodegeneration (HP:0002180); Delayed fine motor development (HP:0010862); Tube feeding (HP:0033454); Constipation (HP:0002019); Failure to thrive (HP:0001508); Hypoplasia of the pons (HP:0012110); Enlarged cisterna magna (HP:0002280); Hypotonia (HP:0001252); Cerebellar vermis hypoplasia (HP:0001320); Intellectual disability, severe (HP:0010864); Hearing impairment (HP:0000365); Plagiocephaly (HP:0001357); Hypoventilation (HP:0002791); Visual impairment (HP:0000505); Absent speech (HP:0001344)"
5 (FEMALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.186_187del (heterozygous) NM_001967.3:c.1161_1166del (heterozygous),"Delayed gross motor development (HP:0002194); Myoclonic seizure (HP:0032794); Dysphagia (HP:0002015); Neurodegeneration (HP:0002180); Delayed fine motor development (HP:0010862); Blindness (HP:0000618); Tube feeding (HP:0033454); Constipation (HP:0002019); Failure to thrive (HP:0001508); Hypoplasia of the pons (HP:0012110); Hypotonia (HP:0001252); Cerebellar vermis hypoplasia (HP:0001320); Intellectual disability, severe (HP:0010864); Hearing impairment (HP:0000365); Plagiocephaly (HP:0001357); Hypoventilation (HP:0002791); Ventriculomegaly (HP:0002119); Spasticity (HP:0001257); Absent speech (HP:0001344); Thin corpus callosum (HP:0033725)"
6 (MALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.481G>T (heterozygous),Delayed gross motor development (HP:0002194); Hypotonia (HP:0001252); Mild global developmental delay (HP:0011342); Arachnoid cyst (HP:0100702); Bilateral tonic-clonic seizure (HP:0002069); Delayed fine motor development (HP:0010862); Delayed speech and language development (HP:0000750)
7 (FEMALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.574G>A (heterozygous),Delayed gross motor development (HP:0002194); Hypotonia (HP:0001252); Tube feeding (HP:0033454); Short corpus callosum (HP:0200012); Cerebral visual impairment (HP:0100704); Severe global developmental delay (HP:0011344); Epileptic spasm (HP:0011097); Absent speech (HP:0001344)
8 (MALE; ),"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)",NM_001967.3:c.641C>A (heterozygous),"Delayed

In [30]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 15 GA4GH phenopackets to the directory phenopackets


In [31]:
PMID = metadata.get_pmid()
builder = HpoaTableBuilder.from_individuals(individuals, created_by)
hpoa_creator = builder.infantile_onset(PMID, 15, 15).autosomal_recessive(PMID).autosomal_dominant(PMID).build()
df = hpoa_creator.get_dataframe()
df

We found a total of 74 unique HPO terms
Extracted disease: Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures (OMIM:620455)


,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0000750,Delayed speech and language development,,,6/6,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0010862,Delayed fine motor development,,,11/11,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0001263,Global developmental delay,,,6/6,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0001256,"Intellectual disability, mild",,,2/2,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0000286,Epicanthus,,,2/2,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0002033,Poor suck,,,1/1,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
73,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0000582,Upslanted palpebral fissure,,,1/1,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
74,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0003593,Infantile onset,,,15/15,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199
75,OMIM:620455,"Neurodevelopmental disorder with hypotonia and speech delay, with or without seizures",HP:0000007,Autosomal recessive inheritance,,,,,,,,PMID:36528028,PCS,ORCID:0000-0002-0736-9199


In [32]:
hpoa_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620455.tab
